# Notebook to generate Figs. 1 & 2

In [ ]:
import numpy as np
import numpy.random as rand
import math
import sys
import os
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
   sys.path.append(nb_dir)
import timeit

from scipy.stats import chi2

%matplotlib notebook
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
# write DT data to file
tMax = 6
ID_list = [kk+1 for kk in range(20)]
randID_list = [1,6,9,13,19]
randID_dict = {}
cond_list = ['baseline', 'additive', 'multiplicative', 'equal']

for randID in randID_list:
    DTopt1_list = []
    DTopt1_list_SE = []
    DTopt2_list = []
    DTopt2_list_SE = []
    Acc_list = []
    Acc_list_STD = []
    AccDict = {}
    PercentileDictOpt1 = {}
    PercentileDictOpt2 = {}
    for ID in ID_list:
        runID = []
        choice = []
        DT = []
        if ID < 17:
            modSys = 'DDMOU'
        else:
            modSys = 'LCA'
        file_to_open = './DataGeneration/ModelMimicry2019_DataGeneration/'+str(modSys)+\
        '_DataGeneration_'+str(randID)+'/'+str(modSys)+'-24Jun2019-'+str(randID)+\
        '-runid-'+str(ID)+'.csv'
        with open(file_to_open, 'r') as infile:
            infile.readline
            for line in infile:
                if not line.startswith('#'): 
                    runID.append(float(line.split(',')[0])) 
                    choice.append(float(line.split(',')[1]))  
                    DT.append(float(line.split(',')[2]))
        
        DTopt1 = np.asarray([DT[kk] for kk in range(len(DT)) if choice[kk]==1 and DT[kk] < tMax])
        DTopt2 = np.asarray([DT[kk] for kk in range(len(DT)) if choice[kk]==0 and DT[kk] < tMax])
        Accuracy = len(DTopt1)/(len(DTopt1)+len(DTopt2))
        AccSTD = np.sqrt(Accuracy*(1-Accuracy)/(len(DTopt1)+len(DTopt2)))
        
        
        print('################################################################################')
        print('random ID = ', randID)
        print('run ID = ', ID)
        print('Total number of trials = ', len(DT))
        print('Total number of trials for analysis (DT <= ' + str(tMax) + ') = ', len(DTopt1)+len(DTopt2))
        print('Number (percentage) of trials removed from dataset (DT > ' + str(tMax) + ') = ', len(DT) - (len(DTopt1)+len(DTopt2)), 
             '('+str(round(((1 - (len(DTopt1)+len(DTopt2))/len(DT))*100),3))+' %)')
        print('DTopt1 Mean = ', np.mean(DTopt1))
        print('DTopt2 Mean = ', np.mean(DTopt2))
        print('###############################################################################')
        
        
        perc10_DTopt1 = np.percentile(DTopt1, 10, axis=0, interpolation='linear')
        perc30_DTopt1 = np.percentile(DTopt1, 30, axis=0, interpolation='linear')
        perc50_DTopt1 = np.percentile(DTopt1, 50, axis=0, interpolation='linear')
        perc70_DTopt1 = np.percentile(DTopt1, 70, axis=0, interpolation='linear')
        perc90_DTopt1 = np.percentile(DTopt1, 90, axis=0, interpolation='linear')
        perc10_DTopt2 = np.percentile(DTopt2, 10, axis=0, interpolation='linear')
        perc30_DTopt2 = np.percentile(DTopt2, 30, axis=0, interpolation='linear')
        perc50_DTopt2 = np.percentile(DTopt2, 50, axis=0, interpolation='linear')
        perc70_DTopt2 = np.percentile(DTopt2, 70, axis=0, interpolation='linear')
        perc90_DTopt2 = np.percentile(DTopt2, 90, axis=0, interpolation='linear')

        DTopt1_list.append(np.mean(DTopt1))
        DTopt1_list_SE.append(np.std(DTopt1)/np.sqrt(len(DTopt1)))
        DTopt2_list.append(np.mean(DTopt2))
        DTopt2_list_SE.append(np.std(DTopt2)/np.sqrt(len(DTopt2)))
        Acc_list.append(Accuracy)
        Acc_list_STD.append(AccSTD)
        AccDict[str(ID)] = Accuracy
        #PercentileDictOpt1[str(ID)] = np.array([perc10_DTopt1, perc30_DTopt1, perc50_DTopt1, perc70_DTopt1, perc90_DTopt1])
        #PercentileDictOpt2[str(ID)] = np.array([perc10_DTopt2, perc30_DTopt2, perc50_DTopt2, perc70_DTopt2, perc90_DTopt2])
    randID_dict[str(randID)] = (DTopt1_list, DTopt1_list_SE, DTopt2_list, DTopt2_list_SE, Acc_list, Acc_list_STD)

In [ ]:
from matplotlib.lines import Line2D

fig = plt.figure(figsize=(12, 18))

chooseCI = 95 # choose confidence interval in percent
if chooseCI == 95:
    factorCI = 1.96 # assumes known standard deviation of Gaussian distribution
else:
    factorCI = 1.0

marker_color = '0.7'
marker_size=13
cap_thick=3
markerSym = ['o', '>', 'v', 'D', 's']
markerCol = ['0.3', '0.45', '0.6', '0.75', '0.9']

nr_row = 5
nr_col = 3

cond_list_xlab = ['base', 'add', 'mult', 'equal'] 

ax1 = plt.subplot2grid((nr_row, nr_col), (0, 0), colspan=1)
#ax1.plot(range(len(cond_list)), DTopt1_list[0:len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    if number == 1:
        ps = 1
    elif number == 6:
        ps=2
    elif number == 9:
        ps=3
    elif number == 13:
        ps=4
    elif number == 19:
        ps=5
    
    """
    if number == 1:
        ps = 1
    elif number == 2:
        ps=2
    elif number == 6:
        ps=3
    elif number == 9:
        ps=4
    elif number == 13:
        ps=5
    elif number == 19:
        ps=6
    """
    yerr = np.asarray(randID_dict[str(number)][1][0:len(cond_list)])*factorCI
    #yerr = np.asarray(DTopt1_list_SE[0:len(cond_list)])*factorCI
    ax1.errorbar(range(len(cond_list)), randID_dict[str(number)][0][0:len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False, label='data set '+str(ps))
    mSymInd += 1
    mColInd += 1
ax1.set_xticks(range(len(cond_list)))
ax1.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax1.xaxis.label.set_size(18)
ax1.yaxis.label.set_size(18)
ax1.tick_params(length=6, width=2, axis='both')
ax1.tick_params(axis='y', labelsize = 14)
#ax1.set_xlim([-0.2, 3.2])
##ax1.set_xlabel('condition')
ax1.set_ylabel(r'$\langle DT_1\rangle$ (s)')
ax1.set_title(r'A: mSOU, opt.1', x=0.52, fontsize=20, weight='bold')
ax1.legend(loc='center left', bbox_to_anchor=(0.1, 1.4), ncol=3, 
           shadow=False, fancybox=True, columnspacing=4,fontsize=20)

ax2 = plt.subplot2grid((nr_row, nr_col), (0, 1), colspan=1, sharey=ax1)
#ax2.plot(range(len(cond_list)), DTopt2_list[0:len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][3][0:len(cond_list)])*factorCI
    #yerr = np.asarray(DTopt1_list_SE[0:len(cond_list)])*factorCI
    ax2.errorbar(range(len(cond_list)), randID_dict[str(number)][2][0:len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax2.set_xticks(range(len(cond_list)))
ax2.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax2.xaxis.label.set_size(18)
ax2.yaxis.label.set_size(18)
ax2.tick_params(length=6, width=2, axis='both')
ax2.tick_params(axis='y', labelsize = 14)
#ax2.set_xlim([-0.2, 3.2])
#ax2.set_ylim([0.35, 0.95])
##ax2.set_xlabel('condition')
ax2.set_ylabel(r'$\langle DT_2\rangle$ (s)')
ax2.set_title('B: mSOU, opt.2', x=0.52, fontsize=20, weight='bold')

ax3 = plt.subplot2grid((nr_row, nr_col), (0, 2), colspan=1)
#ax3.plot(range(len(cond_list)), Acc_list[0:len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][5][0:len(cond_list)])*factorCI
    #yerr = np.asarray(DTopt1_list_SE[0:len(cond_list)])*factorCI
    ax3.errorbar(range(len(cond_list)), randID_dict[str(number)][4][0:len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax3.set_xticks(range(len(cond_list)))
ax3.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax3.xaxis.label.set_size(18)
ax3.yaxis.label.set_size(18)
ax3.tick_params(length=6, width=2, axis='both')
ax3.tick_params(axis='y', labelsize = 14)
#ax3.set_xlim([-0.2, 3.2])
#ax3.set_ylim([0.48, 0.7])
##ax3.set_xlabel('condition')
ax3.set_ylabel(r'$Pr(opt1)$')
ax3.set_title('C: mSOU, choice', x=0.52, fontsize=20, weight='bold')

ax4 = plt.subplot2grid((nr_row, nr_col), (1, 0), colspan=1)
#ax4.plot(range(len(cond_list)), DTopt1_list[4:4+len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][1][4:4+len(cond_list)])*factorCI
    ax4.errorbar(range(len(cond_list)), randID_dict[str(number)][0][4:4+len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax4.set_xticks(range(len(cond_list)))
ax4.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax4.xaxis.label.set_size(18)
ax4.yaxis.label.set_size(18)
ax4.tick_params(length=6, width=2, axis='both')
ax4.tick_params(axis='y', labelsize = 14)
#ax4.set_xlim([-0.2, 3.2])
##ax4.set_xlabel('condition')
ax4.set_ylabel(r'$\langle DT_1\rangle$ (s)')
ax4.set_title('D: mUOU, opt.1', x=0.52, fontsize=20, weight='bold')

ax5 = plt.subplot2grid((nr_row, nr_col), (1, 1), colspan=1, sharey=ax4)
#ax5.plot(range(len(cond_list)), DTopt2_list[4:4+len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][3][4:4+len(cond_list)])*factorCI
    #yerr = np.asarray(DTopt1_list_SE[0:len(cond_list)])*factorCI
    ax5.errorbar(range(len(cond_list)), randID_dict[str(number)][2][4:4+len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax5.set_xticks(range(len(cond_list)))
ax5.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax5.xaxis.label.set_size(18)
ax5.yaxis.label.set_size(18)
ax5.tick_params(length=6, width=2, axis='both')
ax5.tick_params(axis='y', labelsize = 14)
#ax5.set_xlim([-0.2, 3.2])
##ax5.set_xlabel('condition')
ax5.set_ylabel(r'$\langle DT_2\rangle$ (s)')
ax5.set_title('E: mUOU, opt.2', x=0.52, fontsize=20, weight='bold')

ax6 = plt.subplot2grid((nr_row, nr_col), (1, 2), colspan=1)
#ax6.plot(range(len(cond_list)), Acc_list[4:4+len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][5][4:4+len(cond_list)])*factorCI
    #yerr = np.asarray(DTopt1_list_SE[0:len(cond_list)])*factorCI
    ax6.errorbar(range(len(cond_list)), randID_dict[str(number)][4][4:4+len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax6.set_xticks(range(len(cond_list)))
ax6.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax6.xaxis.label.set_size(18)
ax6.yaxis.label.set_size(18)
ax6.tick_params(length=6, width=2, axis='both')
ax6.tick_params(axis='y', labelsize = 14)
#ax6.set_xlim([-0.2, 3.2])
#ax6.set_ylim([0.48, 0.7])
##ax6.set_xlabel('condition')
ax6.set_ylabel(r'$Pr(opt1)$')
ax6.set_title('F: mUOU, choice', x=0.52, fontsize=20, weight='bold')

ax7 = plt.subplot2grid((nr_row, nr_col), (2, 0), colspan=1)
#ax7.plot(range(len(cond_list)), DTopt1_list[8:8+len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][1][8:8+len(cond_list)])*factorCI
    ax7.errorbar(range(len(cond_list)), randID_dict[str(number)][0][8:8+len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax7.set_xticks(range(len(cond_list)))
ax7.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax7.xaxis.label.set_size(18)
ax7.yaxis.label.set_size(18)
ax7.tick_params(length=6, width=2, axis='both')
ax7.tick_params(axis='y', labelsize = 14)
#ax7.set_xlim([-0.2, 3.2])
##ax7.set_xlabel('condition')
ax7.set_ylabel(r'$\langle DT_1\rangle$ (s)')
ax7.set_title('G: mDDM, opt.1', x=0.52, fontsize=20, weight='bold')

ax8 = plt.subplot2grid((nr_row, nr_col), (2, 1), colspan=1, sharey=ax7)
#ax8.plot(range(len(cond_list)), DTopt2_list[8:8+len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][3][8:8+len(cond_list)])*factorCI
    #yerr = np.asarray(DTopt1_list_SE[0:len(cond_list)])*factorCI
    ax8.errorbar(range(len(cond_list)), randID_dict[str(number)][2][8:8+len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax8.set_xticks(range(len(cond_list)))
ax8.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax8.xaxis.label.set_size(18)
ax8.yaxis.label.set_size(18)
ax8.tick_params(length=6, width=2, axis='both')
ax8.tick_params(axis='y', labelsize = 14)
#ax8.set_xlim([-0.2, 3.2])
#ax8.set_ylim([0.44, 0.8])
##ax8.set_xlabel('condition')
ax8.set_ylabel(r'$\langle DT_2\rangle$ (s)')
ax8.set_title('H: mDDM, opt.2', x=0.52, fontsize=20, weight='bold')

ax9 = plt.subplot2grid((nr_row, nr_col), (2, 2), colspan=1)
#ax9.plot(range(len(cond_list)), Acc_list[8:8+len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][5][8:8+len(cond_list)])*factorCI
    #yerr = np.asarray(DTopt1_list_SE[0:len(cond_list)])*factorCI
    ax9.errorbar(range(len(cond_list)), randID_dict[str(number)][4][8:8+len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax9.set_xticks(range(len(cond_list)))
ax9.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax9.xaxis.label.set_size(18)
ax9.yaxis.label.set_size(18)
ax9.tick_params(length=6, width=2, axis='both')
ax9.tick_params(axis='y', labelsize = 14)
#ax9.set_xlim([-0.2, 3.2])
#ax9.set_ylim([0.48, 0.7])
#ax9.set_xlabel('condition')
ax9.set_ylabel(r'$Pr(opt1)$')
ax9.set_title('I: mDDM, choice', x=0.52, fontsize=20, weight='bold')

ax10 = plt.subplot2grid((nr_row, nr_col), (3, 0), colspan=1)
#ax10.plot(range(len(cond_list)), DTopt1_list[12:12+len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][1][12:12+len(cond_list)])*factorCI
    ax10.errorbar(range(len(cond_list)), randID_dict[str(number)][0][12:12+len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax10.set_xticks(range(len(cond_list)))
ax10.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax10.xaxis.label.set_size(18)
ax10.yaxis.label.set_size(18)
ax10.tick_params(length=6, width=2, axis='both')
ax10.tick_params(axis='y', labelsize = 14)
#ax10.set_xlim([-0.2, 3.2])
##ax10.set_xlabel('condition')
ax10.set_ylabel(r'$\langle DT_1\rangle$ (s)')
ax10.set_title('J: pDDM, opt.1', x=0.52, y=1.02, fontsize=20, weight='bold')

ax11 = plt.subplot2grid((nr_row, nr_col), (3, 1), colspan=1, sharey=ax10)
#ax11.plot(range(len(cond_list)), DTopt2_list[12:12+len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][3][12:12+len(cond_list)])*factorCI
    #yerr = np.asarray(DTopt1_list_SE[0:len(cond_list)])*factorCI
    ax11.errorbar(range(len(cond_list)), randID_dict[str(number)][2][12:12+len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax11.set_xticks(range(len(cond_list)))
ax11.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax11.xaxis.label.set_size(18)
ax11.yaxis.label.set_size(18)
ax11.tick_params(length=6, width=2, axis='both')
ax11.tick_params(axis='y', labelsize = 14)
#ax11.set_xlim([-0.2, 3.2])
#ax11.set_ylim([0.6, 0.87])
#ax11.set_xlabel('condition')
ax11.set_ylabel(r'$\langle DT_2\rangle$ (s)')
ax11.set_title('K: pDDM, opt.2', x=0.52, fontsize=20, weight='bold')

ax12 = plt.subplot2grid((nr_row, nr_col), (3, 2), colspan=1)
#ax12.plot(range(len(cond_list)), Acc_list[12:12+len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][5][12:12+len(cond_list)])*factorCI
    #yerr = np.asarray(DTopt1_list_SE[0:len(cond_list)])*factorCI
    ax12.errorbar(range(len(cond_list)), randID_dict[str(number)][4][12:12+len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax12.set_xticks(range(len(cond_list)))
ax12.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax12.xaxis.label.set_size(18)
ax12.yaxis.label.set_size(18)
ax12.tick_params(length=6, width=2, axis='both')
ax12.tick_params(axis='y', labelsize = 14)
#ax12.set_xlim([-0.2, 3.2])
#ax12.set_ylim([0.48, 0.92])
#ax12.set_xlabel('condition')
ax12.set_ylabel(r'$Pr(opt1)$')
ax12.set_title('L: pDDM, choice', x=0.52, fontsize=20, weight='bold')

ax13 = plt.subplot2grid((nr_row, nr_col), (4, 0), colspan=1)
#ax13.plot(range(len(cond_list)), DTopt1_list[16:16+len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][1][16:16+len(cond_list)])*factorCI
    ax13.errorbar(range(len(cond_list)), randID_dict[str(number)][0][16:16+len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax13.set_xticks(range(len(cond_list)))
ax13.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax13.xaxis.label.set_size(18)
ax13.yaxis.label.set_size(18)
ax13.tick_params(length=6, width=2, axis='both')
ax13.tick_params(axis='y', labelsize = 14)
#ax13.set_xlim([-0.2, 3.2])
ax13.set_xlabel('condition')
ax13.set_ylabel(r'$\langle DT_1\rangle$ (s)')
ax13.set_title('M: LCA, opt.1', x=0.52, fontsize=20, weight='bold')

ax14 = plt.subplot2grid((nr_row, nr_col), (4, 1), colspan=1, sharey=ax13)
#ax14.plot(range(len(cond_list)), DTopt2_list[16:16+len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][3][16:16+len(cond_list)])*factorCI
    #yerr = np.asarray(DTopt1_list_SE[0:len(cond_list)])*factorCI
    ax14.errorbar(range(len(cond_list)), randID_dict[str(number)][2][16:16+len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax14.set_xticks(range(len(cond_list)))
ax14.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax14.xaxis.label.set_size(18)
ax14.yaxis.label.set_size(18)
ax14.tick_params(length=6, width=2, axis='both')
ax14.tick_params(axis='y', labelsize = 14)
#ax14.set_xlim([-0.2, 3.2])
#ax14.set_ylim([0.5, 0.71])
ax14.set_xlabel('condition')
ax14.set_ylabel(r'$\langle DT_2\rangle$ (s)')
ax14.set_title('N: LCA, opt.2', x=0.52, fontsize=20, weight='bold')

ax15 = plt.subplot2grid((nr_row, nr_col), (4, 2), colspan=1)
#ax15.plot(range(len(cond_list)), Acc_list[16:16+len(cond_list)], 'ko', ms=10, ls='solid', lw=3)
mSymInd = 0
mColInd = 0
for number in randID_list:
    yerr = np.asarray(randID_dict[str(number)][5][16:16+len(cond_list)])*factorCI
    #yerr = np.asarray(DTopt1_list_SE[0:len(cond_list)])*factorCI
    ax15.errorbar(range(len(cond_list)), randID_dict[str(number)][4][16:16+len(cond_list)], yerr=yerr, 
                 color=markerCol[mColInd], ls='-', lw=2, marker=markerSym[mSymInd], ms=marker_size, mfc=markerCol[mColInd], mec='k', mew=1.5,
                 capsize= 10, capthick=cap_thick, elinewidth=2, ecolor='k', barsabove=False)
    mSymInd += 1
    mColInd += 1
ax15.set_xticks(range(len(cond_list)))
ax15.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax15.xaxis.label.set_size(18)
ax15.yaxis.label.set_size(18)
ax15.tick_params(length=6, width=2, axis='both')
ax15.tick_params(axis='y', labelsize = 20)
#ax15.set_xlim([-0.2, 3.2])
#ax15.set_ylim([0.48, 0.7])
ax15.set_xlabel('condition')
ax15.set_ylabel(r'$Pr(opt1)$')
ax15.set_title('O: LCA, choice', x=0.52, fontsize=20, weight='bold')

for axx in [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11,ax12,ax13,ax14,ax15]:
    if axx in [ax3,ax6,ax9,ax12,ax15]:
        axx.yaxis.set_major_locator(ticker.MultipleLocator(0.05))
        axx.yaxis.set_minor_locator(ticker.MultipleLocator(0.025))         
    else:
        axx.yaxis.set_major_locator(ticker.MultipleLocator(0.2))
        axx.yaxis.set_minor_locator(ticker.MultipleLocator(0.1))
    for axis in ['top','bottom','left','right']:
        axx.spines[axis].set_linewidth(2)
    axx.tick_params('both', length=8, width=2, which='major')
    axx.tick_params('both', length=6, width=1, which='minor')
    axx.tick_params(axis='y', labelsize = 20)
    axx.tick_params(axis='x', labelsize = 17)
    axx.xaxis.label.set_size(26)
    axx.yaxis.label.set_size(26)


fig.tight_layout(rect=[0, 0.02, 1, 0.94])

In [ ]:
#fig.savefig("Comparison_generatedData.eps")#, dpi=150)

In [ ]:
fig1 = plt.figure(figsize=(12, 6))

chooseCI = 95 # choose confidence interval in percent
if chooseCI == 95:
    factorCI = 1.96 # assumes known standard deviation of Gaussian distribution
else:
    factorCI = 1.0

marker_size=18
markerSym = ['s', 'v', '+', 'x']
markerCol = ['0.3', '0.45', '0.6', '0.75']

nr_row = 1
nr_col = 2
DTindex = 0 #0 -> DT1, #2->DT2
ax1 = plt.subplot2grid((nr_row, nr_col), (0, 0), colspan=1)
diffDT_mSOU = []
diffDT_mUOU = []
diffDT_mDDM = []
diffDT_pDDM = []
diffDT_LCA = []
for number in randID_list:
    diffDT_mSOU.append(randID_dict[str(number)][DTindex][1]-randID_dict[str(number)][DTindex][2])
    diffDT_mUOU.append(randID_dict[str(number)][DTindex][5]-randID_dict[str(number)][DTindex][6])
    diffDT_mDDM.append(randID_dict[str(number)][DTindex][9]-randID_dict[str(number)][DTindex][10])
    diffDT_pDDM.append(randID_dict[str(number)][DTindex][13]-randID_dict[str(number)][DTindex][14])
    diffDT_LCA.append(randID_dict[str(number)][DTindex][17]-randID_dict[str(number)][DTindex][18])
    
ax1.plot([0.5,5.5], [0,0], 
         color='orange', ls='--', lw=3)
ax1.plot([1,2,3,4,5], diffDT_mSOU, 
         color=markerCol[0], ls='-', lw=3.5, marker=markerSym[0], ms=marker_size, mfc=markerCol[0], mec='k', mew=1.5,
         label='mSOU')
ax1.plot([1,2,3,4,5], diffDT_mUOU, 
         color=markerCol[1], ls='-', lw=3.5, marker=markerSym[1], ms=marker_size, mfc=markerCol[1], mec='k', mew=1.5,
         label='mUOU')
ax1.plot([1,2,3,4,5], diffDT_mDDM, 
         color=markerCol[2], ls='-', lw=3.5, marker=markerSym[2], ms=marker_size, mfc=markerCol[2], mec='k', mew=3,
         label='mDDM')
#ax1.plot([1,2,3,4,5,6], diffDT_pDDM, 
#         color=markerCol[3], ls='-', lw=3.5, marker=markerSym[3], ms=marker_size, mfc=markerCol[3], mec='k', mew=1.5,
#         label='pDDM')
ax1.plot([1,2,3,4,5], diffDT_LCA, 
         color=markerCol[3], ls='-', lw=3.5, marker=markerSym[3], ms=marker_size, mfc=markerCol[3], mec='k', mew=3,
         label='LCA')

ax1.set_xticks(range(1,7))
#ax1.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax1.xaxis.label.set_size(26)
ax1.yaxis.label.set_size(26)
ax1.tick_params(length=6, width=2, axis='both')
ax1.tick_params(axis='both', labelsize = 24)
ax1.set_xlim([0.5, 5.5])
ax1.set_xlabel('data set identifier')
ax1.set_ylabel(r'$\langle DT_1\rangle_{add}-\langle DT_1\rangle_{mul}$ (s)')
ax1.set_title(r'A', x=0.1, fontsize=30, weight='bold')
ax1.legend(loc='center left', bbox_to_anchor=(0.4, 1.3), ncol=2, 
           shadow=False, fancybox=True, columnspacing=4,fontsize=22)
#ax1.legend(loc='upper left', ncol=2, 
#           shadow=False, fancybox=True, columnspacing=2,fontsize=22)

DTindex = 2 #0 -> DT1, #2->DT2
ax2 = plt.subplot2grid((nr_row, nr_col), (0, 1), colspan=1)
diffDT_mSOU = []
diffDT_mUOU = []
diffDT_mDDM = []
diffDT_pDDM = []
diffDT_LCA = []
for number in randID_list:
    diffDT_mSOU.append(randID_dict[str(number)][DTindex][1]-randID_dict[str(number)][DTindex][2])
    diffDT_mUOU.append(randID_dict[str(number)][DTindex][5]-randID_dict[str(number)][DTindex][6])
    diffDT_mDDM.append(randID_dict[str(number)][DTindex][9]-randID_dict[str(number)][DTindex][10])
    diffDT_pDDM.append(randID_dict[str(number)][DTindex][13]-randID_dict[str(number)][DTindex][14])
    diffDT_LCA.append(randID_dict[str(number)][DTindex][17]-randID_dict[str(number)][DTindex][18])
    
ax2.plot([0.5,5.5], [0,0], 
         color='orange', ls='--', lw=3)
ax2.plot([1,2,3,4,5], diffDT_mSOU, 
         color=markerCol[0], ls='-', lw=3.5, marker=markerSym[0], ms=marker_size, mfc=markerCol[0], mec='k', mew=1.5,
         label='mSOU')
ax2.plot([1,2,3,4,5], diffDT_mUOU, 
         color=markerCol[1], ls='-', lw=3.5, marker=markerSym[1], ms=marker_size, mfc=markerCol[1], mec='k', mew=1.5,
         label='mUOU')
ax2.plot([1,2,3,4,5], diffDT_mDDM, 
         color=markerCol[2], ls='-', lw=3.5, marker=markerSym[2], ms=marker_size, mfc=markerCol[2], mec='k', mew=3,
         label='mDDM')
#ax2.plot([1,2,3,4,5,6], diffDT_pDDM, 
#         color=markerCol[3], ls='-', lw=3.5, marker=markerSym[3], ms=marker_size, mfc=markerCol[3], mec='k', mew=1.5,
#         label='pDDM')
ax2.plot([1,2,3,4,5], diffDT_LCA, 
         color=markerCol[3], ls='-', lw=3.5, marker=markerSym[3], ms=marker_size, mfc=markerCol[3], mec='k', mew=3,
         label='LCA')

ax2.set_xticks(range(1,7))
#ax2.set_xticklabels(cond_list_xlab, rotation='horizontal', fontsize=13)
ax2.xaxis.label.set_size(26)
ax2.yaxis.label.set_size(26)
ax2.tick_params(length=6, width=2, axis='both')
ax2.tick_params(axis='both', labelsize = 24)
ax2.set_xlim([0.5, 5.5])
ax2.set_xlabel('data set identifier')
ax2.set_ylabel(r'$\langle DT_2\rangle_{add}-\langle DT_2\rangle_{mul}$ (s)')
ax2.set_title(r'B', x=0.1, fontsize=30, weight='bold')

fig1.tight_layout(rect=[0.005, 0.02, 0.995, 0.83])

In [ ]:
#fig1.savefig("DTadd_minus_DTmul_generatedData.eps")